In [5]:
import torch
from torchvision import transforms
from torchvision import models
from PIL import Image
import cv2

In [6]:
import torch.nn as nn

# 1. ResNet50 모델 정의
model = models.resnet50(weights=None)

# 2. 변경된 분류기 정의하기
model.fc = nn.Sequential(
    nn.Linear(model.fc.in_features, 1024),  # 더 큰 크기의 레이어
    nn.BatchNorm1d(1024),  # Batch Normalization 추가
    nn.ReLU(),
    nn.Dropout(0.4),  # Dropout 비율 조정
    nn.Linear(1024, 512),
    nn.BatchNorm1d(512),  # Batch Normalization 추가
    nn.ReLU(),
    nn.Dropout(0.4),
    nn.Linear(512, 34),
)

# 3. 가중치 로드
state_dict = torch.load('model_food/RESNET50.pth', map_location=torch.device('cpu'))
model.load_state_dict(state_dict)

# 4. 평가 모드로 설정
model.eval()

C:\Users\user\AppData\Local\Temp\ipykernel_15964\125572531.py:20: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load('model_food/RESNET50.pth', map_locati

FileNotFoundError: [Errno 2] No such file or directory: 'model_food/RESNET50.pth'

In [2]:
org_size = (256, 256)
# 이미지 전처리 (모델 학습시 사용했던 전처리 방법을 그대로 사용해야 함)
test_transform = transforms.Compose([
    transforms.Resize(org_size),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

In [3]:


# 웹캠 열기
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # OpenCV 이미지를 PIL 이미지로 변환
    img = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

    # 이미지 전처리
    img_tensor = test_transform(img).unsqueeze(0)  # 배치 차원 추가

    # 예측
    with torch.no_grad():
        output = model(img_tensor)
        _, predicted = torch.max(output, 1)
        label = predicted.item()

    # 예측한 레이블을 화면에 표시
    cv2.putText(frame, f"Predicted Label: {label}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

    # 웹캠 화면 출력
    cv2.imshow("Webcam", frame)

    # 'q'를 누르면 종료
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

NameError: name 'cv2' is not defined